# Set Up Environment

In [ ]:
!pip install transformers datasets peft

In [ ]:
!pip install bitsandbytes==0.43.2

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-7b-instruct"

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16  # Use float16 for computation
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically distribute layers across GPU/CPU
)

# Ensure the tokenizer's pad token is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load and Preprocess the Dataset:

In [ ]:
import pandas as pd
from datasets import Dataset

# Load dataset
df = pd.read_csv("cleaned_dataset.csv")
dataset = Dataset.from_pandas(df)

# Tokenization
def preprocess_function(examples):
    inputs = [f"Client: {text}\nTherapist:" for text in examples["client"]]
    targets = [text for text in examples["therapist"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, truncation=True, padding="max_length", max_length=512
    )
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

#  Fine-Tune Using LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],  # Target attention layers
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Set Training Arguments:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none"
)

# training the model:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-11-4ee6e095f1a9>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=57, training_loss=249.99883497807016, metrics={'train_runtime': 2808.5272, 'train_samples_per_second': 0.681, 'train_steps_per_second': 0.02, 'total_flos': 3.706082893234176e+16, 'train_loss': 249.99883497807016, 'epoch': 2.852664576802508})

# save the finetuned model

In [ ]:
# Save the model
model.save_pretrained("./fine_tuned_falcon")
tokenizer.save_pretrained("./fine_tuned_falcon")

('./fine_tuned_falcon/tokenizer_config.json',
 './fine_tuned_falcon/special_tokens_map.json',
 './fine_tuned_falcon/tokenizer.json')

In [ ]:
# Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained('/content/drive/MyDrive/fine_tuned_falcon')
tokenizer.save_pretrained('/content/drive/MyDrive/fine_tuned_falcon')

Mounted at /content/drive


NameError: name 'model' is not defined

# generate responses

In [ ]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_response(input_text):
    prompt = f"Client: {input_text}\nTherapist:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=01.5,
        top_p=0.9,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Therapist:")[1].strip()

# Test inputs
test_input = "I've been feeling disconnected from my friends lately. what should i do?"
response = generate_response(test_input)
print(f"Input: {input_text}")
print(f"Response: {response}\n")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Input: I've been feeling disconnected from my friends lately. what should i do?
Response: It's important to reach out to your friends and communicate how you're feeling. Consider scheduling a time to talk with them or even suggesting a group therapy session. It's also helpful to practice self-care and engage in activities that bring



In [ ]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import torch

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model and tokenizer
model_path = '/content/drive/MyDrive/fine_tuned_falcon'
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model with 4-bit quantization
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=quantization_config).to(device)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_response(input_text, max_length=150):
    prompt = f"Client: {input_text}\nTherapist:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,  # Set max_length to control the total output length
            temperature=1.5,
            top_p=0.95,
            repetition_penalty=1.2
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Therapist:")[1].strip()

In [ ]:
# Test input
test_input = "I feel like I’m not good enough for my family."
response = generate_response(test_input)
print(response)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


It’s understandable to feel that way sometimes. Can you tell me more about why you feel this way?
